# Benchmark of Keysight 34465A

We benchmark the acquisition of N voltages from a QDac on a Keysight DMM. Two acquisition modes are tested; the "classical" set voltage/get voltage and a set voltage / send software trigger method.

The former method is cast as a standard QCoDeS measurement, whereas the latter is just the raw commands. If anything, this should add a bit of extra overhead to the first method.

The QDac sweeps a voltage from 0 V to 1 V.

Conclusion: <span style="color:red">there is **no** speed-up</span>.

In [1]:
import qcodes as qc
import numpy as np
import time
from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
from qcodes.instrument_drivers.QDev.QDac import QDac

In [2]:
dmm = Keysight_34465A('dmm', 'TCPIP0::K-000000-00000.local::hislip0::INSTR')

Connected to: Keysight Technologies 34465A (serial:MY54505281, firmware:A.02.14-02.40-02.14-00.49-02-01) in 7.14s


In [3]:
qdac = QDac('qdac', 'ASRL7::INSTR')

## Pre-benchmark setting

In [4]:
ap_time = 1e-3
dmm.NPLC(0.06)  # aperture time of 1 ms
dmm.range(1)  # Voltage range of 1 V
qdac.ch01_v(0)

## N = 1000, set-get

In [5]:
# prepare the DMM for get-set mode
dmm.trigger_source('IMM')
dmm.sample_count(1)
dmm.trigger_count(1)

In [6]:
%%timeit

N = 1000

# remember to turn off the display for better speed
dmm.display_text('Running set-get for {} points'.format(N))

loop = qc.Loop(qdac.ch01_v.sweep(0, 1, num=N)).each(dmm.volt)
data = loop.get_data_set(name='N_{}_setget'.format(N))
_ = loop.run()  # run the loop

dmm.display_clear()


Started at 2017-09-08 11:33:51
DataSet:
   location = 'data/2017-09-08/#013_N_1000_setget_11-33-51'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | qdac_ch01_v_set | ch01_v       | (1000,)
   Measured | dmm_volt        | volt         | (1000,)
Finished at 2017-09-08 11:34:23
Started at 2017-09-08 11:34:23
DataSet:
   location = 'data/2017-09-08/#014_N_1000_setget_11-34-23'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | qdac_ch01_v_set | ch01_v       | (1000,)
   Measured | dmm_volt        | volt         | (1000,)
Finished at 2017-09-08 11:34:55
Started at 2017-09-08 11:34:55
DataSet:
   location = 'data/2017-09-08/#015_N_1000_setget_11-34-55'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | qdac_ch01_v_set | ch01_v       | (1000,)
   Measured | dmm_volt        | volt         | (1000,)
Finished at 2017-09-08 11:35:27
Started at 2017-09-08 11:35:27
DataSet:
   location = 'data/2017-09-08/#016_N_1000_setget_1

## N = 1000, set-software triggered

In [9]:
def software_triggered_readings(N: int) -> np.array:
    """
    Try to get N measurement points
    """
    
    dmm.display_text('Sending {} software triggers'.format(N))
    
    dmm.trigger_source('BUS')
    dmm.sample_count(1)  # samples per trigger
    dmm.trigger_count(N)  # number of triggers to wait for
    dmm.init_measurement()
    
    qdac_setpoints = np.linspace(0, 1, N)
    
    for n in range(N):
        qdac.ch01_v(qdac_setpoints[n])
        dmm.write('*TRG')
        time.sleep(dmm.sample_timer_minimum())  # Wait to avoid losing triggers
    
    # even if we are losing triggers, can ABORt the measurement
    # and then fetch whatever data the DMM holds
    dmm.abort_measurement()  
    raw_vals = dmm.ask('FETCH?')
    dmm.display_clear()

    numvals = np.array(list(map(float, raw_vals.split(','))))
    
    print('Requested {} samples, got {}.'.format(N, len(numvals)))
    
    return numvals

In [8]:
%%timeit

data = software_triggered_readings(1000)

Requested 1000 samples, got 1000.
Requested 1000 samples, got 1000.
Requested 1000 samples, got 1000.
Requested 1000 samples, got 1000.
Requested 1000 samples, got 1000.
Requested 1000 samples, got 1000.
Requested 1000 samples, got 1000.
Requested 1000 samples, got 1000.
32.2 s ± 387 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Results summary

**N = 1000**

Standard set-get: 32.2 s

Software triggered: 32.2 s